## Ablation - modify (a,b), (c,d) to (a,d) and (c,b) 

In [1]:
import pandas as pd
import numpy as np

In [13]:
import random

In [2]:
data_path = "../datasets/fb15k-237/"

In [3]:
relations_df = pd.read_csv(f"{data_path}/derived/relations_with_ids.csv")

In [4]:
freebase_vs_wiki_df =  pd.read_csv(f"{data_path}/fb_wiki_mapping.tsv", sep='\t')

In [5]:
triplets_with_src_dst_wiki_ids_df = pd.read_csv(f"{data_path}/derived/triplets_with_src_dst_wiki_ids.csv")

In [82]:
# add test split with empty labels
df_test = pd.read_csv(f"{data_path}/derived/triplets_with_src_dst_wiki_ids.test.csv")

In [87]:
# df = triplets_with_src_dst_wiki_ids_df
df = df_test

In [88]:
n = len(df)
num_errors = int(0.1*n)

In [99]:
df.info()
df.timestamp.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18286 entries, 0 to 18285
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   src_freebase_id   18286 non-null  object
 1   relation          18286 non-null  object
 2   dest_freebase_id  18286 non-null  object
 3   timestamp         18286 non-null  object
 4   relation_name     18286 non-null  object
 5   relation_id       18286 non-null  int64 
 6   freebase_id_x     18286 non-null  object
 7   label_x           18286 non-null  object
 8   src_wikidata_id   18286 non-null  int64 
 9   freebase_id_y     18286 non-null  object
 10  label_y           18286 non-null  object
 11  dest_wikidata_id  18286 non-null  int64 
 12  triple_id         18286 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 1.8+ MB


timestamp
2025-01-02    18286
Name: count, dtype: int64

In [89]:
triple_ids = list(df.triple_id.values)

In [90]:
# permute triple_ids and select top 2*num_errors
# rather select small 
selected_edges = random.sample(triple_ids, 2*num_errors)

In [91]:
perturbed_rows_dicts = []
for i in range(num_errors):
    edge_ab = int(selected_edges[2*i])
    edge_cd = int(selected_edges[2*i+1])
    if i < 5:
        print(f"swapping {edge_ab} and {edge_cd}")

    row_ab = df.loc[df['triple_id']==edge_ab]
    row_cd = df.loc[df['triple_id']==edge_cd]

    row_ab_dict = row_ab.to_dict(orient='records')[0]
    row_cd_dict = row_cd.to_dict(orient='records')[0]

    a = row_ab_dict['src_wikidata_id']
    b = row_ab_dict['dest_wikidata_id']

    c = row_cd_dict['src_wikidata_id']
    d = row_cd_dict['dest_wikidata_id']

    if i < 5:
        print(f"{a}->{b} , {c}->{d}")

    row_ad_new = row_ab_dict.copy()
    row_ad_new['dest_wikidata_id'] = d

    row_cb_new = row_cd_dict.copy()
    row_cb_new['dest_wikidata_id'] = b

    perturbed_rows_dicts.append(row_ad_new)
    perturbed_rows_dicts.append(row_cb_new)
    

swapping 279013 and 290392
170564->739633 , 49088->59085
swapping 283973 and 277823
467362->43445 , 229375->435789
swapping 277596 and 276593
179191->104358 , 23814->178966
swapping 276972 and 274896
326827->801 , 728488->506885
swapping 278134 and 280280
283490->843 , 1338877->189068


In [70]:
# Swap endpoints for edges of same type? 


In [92]:
df_errors = pd.DataFrame.from_dict(perturbed_rows_dicts)

In [93]:
len(df_errors)

3656

In [94]:
df_clean = df[~df['triple_id'].isin(selected_edges)]

In [95]:
len(df_clean)

14630

In [ ]:
# For train dataset
df_errors['inaccurate'] = 1
df_clean['inaccurate'] = 0

df_revised = pd.concat([df_clean, df_errors])

In [75]:
df_revised.to_csv(f"{data_path}/exp2/triplets_with_src_dst_wiki_ids.csv", index=False)

In [96]:
# For test dataset
df_errors['inaccurate'] = 1
df_clean['inaccurate'] = 0

df_revised = pd.concat([df_clean, df_errors])

/var/folders/9f/_45_cttj01b5lmh5x5njvlp00000gn/T/ipykernel_95850/2009358644.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['inaccurate'] = 0


In [97]:
df_revised.to_csv(f"{data_path}/exp2/triplets_with_src_dst_wiki_ids.test.csv", index=False)

In [ ]:
# Evaluation on test set, do the same perturbation
# for them label to be empty per guidelines - https://kumo.ai/docs/examples/fraud-chargeback-abuse/

In [ ]:
# permute as earlier
